In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import (
    NoSuchElementException,
    InvalidSessionIdException,
    TimeoutException,
)
from selenium.webdriver.firefox.options import Options as FirefoxOptions

# from selenium.webdriver import Chrome
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time

In [ ]:
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument("start-maximized")

driver = webdriver.Firefox(options=firefox_options)

In [ ]:
keys = ["politik"]

import os
for key in keys:
    url = f"https://www.sinarharian.com.my/{key}"
    print(f"crawl - {url}")
    driver.get(url)
    time.sleep(10)
    
    n = 0
    n_error = 0
    while True:  
        try:
            articles = "//div[@class='article-title']//a"
            link_articles = []
            for item in driver.find_elements("xpath", articles):
                link_articles.append(item.get_attribute("href"))

            with open(f'output/sinar_harian_link_{os.path.basename(url.rstrip("/"))}.txt', 'w+') as f:
                for line in link_articles:
                    f.write(f"{line}\n")
                
            more = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='more-articles']"))
                )

            if n > 0:
                element_to_click = driver.find_element("xpath", ".//button[@class='more-articles']")
                element_to_click.location_once_scrolled_into_view
            time.sleep(2)
            more.click()
            #print("continue scroll")
            
            n+=1
        except TimeoutException:
            break
        except Exception as e:
            if n_error > 20:
                break
                
            if "could not be scrolled into view" in str(e):
                time.sleep(5)
                element_to_click = driver.find_element("xpath", ".//button[@class='more-articles']")
                element_to_click.location_once_scrolled_into_view
                time.sleep(5)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                # print("error but scroll")
            else:
                n_error += 1
            continue
